heart failure from https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data

In [1]:
import numpy as np
import pandas as pd
import ripser
from sklearn import metrics
from persim.persistent_entropy import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from utils import Compute_Alpha, Parrallel_compute, Data_list

In [2]:
df = pd.read_csv("../Data/heart_failure_clinical_records_dataset.csv")
df.shape

(299, 13)

In [3]:
columns_to_scale = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium']
scaler = StandardScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

X = df.drop(columns=['DEATH_EVENT', 'time'])
y = df['DEATH_EVENT']

print(X.shape)

(299, 11)


In [4]:
data_list = Data_list(X)
len(data_list)

299

In [7]:
results = Parrallel_compute(data_list)

平行運算時間:  0.7255651950836182


In [9]:
X['topo_feature'] = [x.tolist()[0][0] if isinstance(x, np.ndarray) else x for x in results]
X['topo_feature'] = scaler.fit_transform(X[['topo_feature']])
X.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,topo_feature
0,1.192945,0,0.000166,0,-1.530560,1,1.681648e-02,0.490057,-1.504036,1,0,0.309488
1,-0.491279,0,7.514640,0,-0.007077,0,7.535660e-09,-0.284552,-0.141976,1,0,-4.616973
2,0.350833,0,-0.449939,0,-1.530560,0,-1.038073e+00,-0.090900,-1.731046,1,1,-0.470523
3,-0.912335,1,-0.486071,0,-1.530560,0,-5.464741e-01,0.490057,0.085034,1,0,-0.095745
4,0.350833,1,-0.435486,1,-1.530560,0,6.517986e-01,1.264666,-4.682176,0,0,-2.633149


In [11]:
X_final = X.iloc[:, :11].to_numpy()
print(X_final.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.5, random_state=42)

log_reg = LogisticRegression(max_iter = 1000)
log_reg.fit(X_train, y_train)

training_prediction = log_reg.predict(X_train)
test_prediction = log_reg.predict(X_test)

print("Precision, Recall, Confusion matrix, in training\n")
print(metrics.confusion_matrix(y_train, training_prediction))
print(metrics.accuracy_score(y_train, training_prediction))

(299, 11)
Precision, Recall, Confusion matrix, in training

[[105   3]
 [ 25  16]]
0.8120805369127517


In [12]:
X_final = X.to_numpy()
print(X_final.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.5, random_state=42)

log_reg = LogisticRegression(max_iter = 1000)
log_reg.fit(X_train, y_train)

training_prediction = log_reg.predict(X_train)
test_prediction = log_reg.predict(X_test)

print("Precision, Recall, Confusion matrix, in training\n")
print(metrics.confusion_matrix(y_train, training_prediction))
print(metrics.accuracy_score(y_train, training_prediction))

(299, 12)
Precision, Recall, Confusion matrix, in training

[[105   3]
 [ 22  19]]
0.8322147651006712
